In [ ]:
import os
import sys
import glob
import time
import datetime
import numpy as np
%pylab inline


plt.style.use('ggplot')

import matplotlib.pyplot as plt


try:
    import sqlite3
except ImportError:
    print ('you need sqlite3 installed to use this program')
    sys.exit(0)


def encode_string(s):
    """
    Simple utility function to make sure a string is proper
    to be used in a SQLite query
    (different than posgtresql, no N to specify unicode)
    EXAMPLE:
      That's my boy! -> 'That''s my boy!'
    """
    return "'"+s.replace("'","''")+"'"




In [ ]:
import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display

In [ ]:
# requires extra forwardslash, //<hostname/<path>/....
metadata_eng = create_engine('sqlite:///./data/track_metadata.db')
lastfm_tag_eng = create_engine('sqlite:///./data/lastfm_tags.db')



In [ ]:
metadata = pd.read_sql_query('SELECT title, song_id, artist_name, duration, year FROM songs', metadata_eng)


In [ ]:
lastfm = pd.read_sql_query('SELECT * FROM sqlite_master where type="table"', lastfm_tag_eng)
lastfm



In [ ]:
user = pd.read_csv('./data/train_triplets.txt',  
                   delim_whitespace=True,
                   header=None)
user.columns = ['user', 'song', 'play_count']

In [ ]:
user.head()

In [ ]:
pop_songs = user.groupby("song").sum().sort_values("play_count", ascending=False)

In [ ]:
json_helper = lambda x: x.to_json(orient="records")[1:-1]

In [ ]:
pop_songs.head(20).reset_index()

In [ ]:
user.head().to_dict(orient="records")

In [ ]:
user.shape

In [ ]:
gb = user.groupby(["user", "song"]).sum()

In [ ]:
gb.shape

In [ ]:
g = gb.play_count.groupby(level=0, group_keys=False)

In [ ]:
%%time 
res = g.apply(lambda x: x.nlargest(2))

In [ ]:
get_topsongs = lambda x, y: x[x.user == y].song.values



In [ ]:
res.shape

In [ ]:
res.to_csv("top_two_tracks_per_user.csv")


In [ ]:
resset = pd.DataFrame(res).reset_index().groupby("user")['song'].apply(list)


In [ ]:
res_set = pd.DataFrame(resset).reset_index()

In [ ]:
import json

In [ ]:
j = res_set.to_json(orient='records')

In [ ]:
with open("top_two_songs_dict.json", "w") as f:
    json.dump(j, f)

In [ ]:
fives = g.apply(lambda x: x.nlargest(5))

In [ ]:
fives = pd.read_csv('./data/top_five_tracks_per_user.csv')

In [ ]:
gb = fives.groupby(['user'])



In [ ]:
%%time
songset = pd.DataFrame(gb['song'].apply(list)).reset_index()

In [ ]:
import json

In [ ]:
j = songset.to_json(orient="records")

In [ ]:
with open("top_songs_dict.json", "w") as f:
    json.dump(j, f)

In [ ]:
tens = pd.read_csv('./data/top_ten_tracks_per_user.csv')

In [ ]:

songset = pd.DataFrame(tens.groupby("user")['song'].apply(list)).reset_index()

In [ ]:
j = songset.to_json(orient="records")

In [ ]:
with open("top_ten_songs_dict.json", "w") as f:
    json.dump(j, f)

In [ ]:
with open("./data/top_fives_songs_dict.json", "r") as f:
    j = json.load(f)

In [ ]:
j